## Install TensorTrade

In [2]:
#!python -m pip install -e .. -U

## Setup Data Fetching




In [3]:
import pandas as pd

data = pd.read_csv('btc_data_train.csv')
test_data = pd.read_csv('btc_data_test.csv')

In [4]:
data.head()

,date,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume
0,7/1/2017 11:00,2505.56,2513.38,2495.12,2509.17,287000.32
1,7/1/2017 12:00,2509.17,2512.87,2484.99,2488.43,393142.50
2,7/1/2017 13:00,2488.43,2488.43,2454.40,2454.43,693254.01
3,7/1/2017 14:00,2454.43,2473.93,2450.83,2459.35,712864.80
4,7/1/2017 15:00,2459.35,2475.00,2450.00,2467.83,682105.41


## Create features with the data module

In [5]:
from tensortrade.data import Node, Module, DataFeed, Stream, Select


def rsi(price: Node, period: float):
    r = price.diff()
    upside = r.clamp_min(0).abs()
    downside = r.clamp_max(0).abs()
    rs = upside.ewm(alpha=1 / period).mean() / downside.ewm(alpha=1 / period).mean()
    return 100*(1 - (1 + rs) ** -1)


def macd(price: Node, fast: float, slow: float, signal: float) -> Node:
    fm = price.ewm(span=fast, adjust=False).mean()
    sm = price.ewm(span=slow, adjust=False).mean()
    md = fm - sm
    signal = md - md.ewm(span=signal, adjust=False).mean()
    return signal


features = []
for c in data.columns[1:]:
    s = Stream(list(data[c])).rename(data[c].name)
    features += [s]

btc_close = Select("BTC:close")(*features)

features += [
    rsi(btc_close, period=24).rename("BTC:rsi"),
    macd(btc_close, fast=10, slow=50, signal=5).rename("BTC:macd"),
]

feed = DataFeed(features)
feed.compile()



features_test = []
for c in test_data.columns[1:]:
    s = Stream(list(test_data[c])).rename(test_data[c].name)
    features_test += [s]

btc_close = Select("BTC:close")(*features_test)

features_test += [
    rsi(btc_close, period=24).rename("BTC:rsi"),
    macd(btc_close, fast=10, slow=50, signal=5).rename("BTC:macd"),
]

feed_test = DataFeed(features_test)
feed_test.compile()

In [6]:
feed.next()
feed.reset()

feed_test.next()
feed_test.reset()

C:\Users\suuser\PycharmProjects\tensortrade\tensortrade\data\stream\node.py:933: RuntimeWarning: invalid value encountered in double_scalars
  v = (w[::-1] * x).sum() / w.sum()


## Setup Trading Environment

In [7]:
from tensortrade.exchanges import Exchange
from tensortrade.exchanges.services.execution.simulated import execute_order
from tensortrade.data import Stream, DataFeed, Module
from tensortrade.instruments import USD, BTC, ETH
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.environments import TradingEnvironment
from tensortrade.rewards import RiskAdjustedReturns
import copy

coinbase = Exchange("coinbase", service=execute_order)(
    Stream(list(data["BTC:close"])).rename("USD-BTC")
)

test_coinbase = Exchange("coinbase", service=execute_order)(
    Stream(list(test_data["BTC:close"])).rename("USD-BTC")
)

portfolio = Portfolio(USD, [
    Wallet(coinbase, 10000 * USD),
    Wallet(coinbase, 0 * BTC),
])

test_portfolio = Portfolio(USD, [
    Wallet(test_coinbase, 10000 * USD),
    Wallet(test_coinbase, 0 * BTC),
])

env = TradingEnvironment(
    feed=feed,
    portfolio=portfolio,
    use_internal=False,
    action_scheme="simple",
    reward_scheme=RiskAdjustedReturns(return_algorithm = 'sortino', window_size = 24),
    window_size=24
)

test_env = TradingEnvironment(
    feed=feed_test,
    portfolio=test_portfolio,
    use_internal=False,
    action_scheme="simple",
    reward_scheme=RiskAdjustedReturns(return_algorithm = 'sortino', window_size = 24),
    window_size=24
)


## Example Data Feed Observation

Even though this observation contains data from the internal data feed, since `use_internal=False` this data will not be provided as input to the observation history. The data that will be added to observation history of the environment will strictly be the nodes that have been included into the data feed that has been provided as a parameter to the trading environment.

In [8]:
env.feed.next()



{'BTC:open': 2505.56,
 'BTC:high': 2513.38,
 'BTC:low': 2495.12,
 'BTC:close': 2509.17,
 'BTC:volume': 287000.32,
 'BTC:rsi': nan,
 'BTC:macd': -1.0105496686365867e-13,
 'coinbase:/USD-BTC': 2509.17,
 'coinbase:/USD:/free': 10000.0,
 'coinbase:/USD:/locked': 0.0,
 'coinbase:/USD:/total': 10000.0,
 'coinbase:/BTC:/free': 0.0,
 'coinbase:/BTC:/locked': 0.0,
 'coinbase:/BTC:/total': 0.0,
 'coinbase:/BTC:/worth': 0.0,
 'net_worth': 10000.0}

In [9]:
import tensortrade.agents.a2c_lstm_agent
from tensortrade.agents.a2c_lstm_agent import A2C_LSTM_Agent
import importlib
importlib.reload(tensortrade.agents.a2c_lstm_agent)

#agent = A2C_LSTM_Agent(env, validation_feed=feed_test)

#agent.train(n_steps=data.shape[0], batch_size=24*7, train_end=0.3, n_episodes=500, save_path="D:/Users/suuser/Desktop/test_results")



<module 'tensortrade.agents.a2c_lstm_agent' from 'C:\\Users\\suuser\\PycharmProjects\\tensortrade\\tensortrade\\agents\\a2c_lstm_agent.py'>

In [ ]:
import tensortrade.agents.a2c_agent
from tensortrade.agents.a2c_agent import A2CAgent
import importlib
importlib.reload(tensortrade.agents.a2c_agent)

env.set_feed(feed)
env.compile()
env.portfolio.reset()

agent = A2CAgent(env, test_env=test_env)
print('starting')
agent.train(n_steps=data.shape[0], batch_size=24*7, train_end=0.3, n_episodes=500, save_path="D:/Users/suuser/Desktop/a2c_test_results")




starting
====      AGENT ID: 12721a4f-c530-4fa3-ba51-e156e4d8d099      ====
[10000.00 USD, 0.00000000 BTC]
====      TRAIN EPISODE ID (1/500): f49b5c0c-8fb1-442c-a148-584ad89f2b2e      ====
step 0/21925
[10000.00 USD, 0.00000000 BTC]
10000.0
exchange: 2509.17
step 24/21925
[4327.31 USD, 2.21744805 BTC]
9759.17074328
exchange: 2449.6
step 48/21925
[9577.26 USD, 0.10318341 BTC]
9835.455846843
exchange: 2502.3
step 72/21925
[2534.92 USD, 2.81601764 BTC]
9898.6090073652
exchange: 2614.93
step 96/21925
[8184.44 USD, 0.56118401 BTC]
9627.838944760599
exchange: 2572.06
step 120/21925
[3.50 USD, 3.65512001 BTC]
9517.7042836298
exchange: 2602.98
step 144/21925
[2329.76 USD, 2.70552215 BTC]
9245.0746154
exchange: 2556.0
step 168/21925
[889.27 USD, 3.21368308 BTC]
8949.99058541
exchange: 2508.25
step 192/21925
[5961.08 USD, 1.09581438 BTC]
8759.0666890292
exchange: 2553.34
step 216/21925
[5211.87 USD, 1.31650854 BTC]
8496.4008262752
exchange: 2494.88
step 240/21925
[8359.02 USD, 0.00035060 BTC]
8